In [1]:
!pip install openai==1.50.2
!pip install langchain==0.2.16
!pip install langchain-community==0.2.17
!pip install langchain-openai==0.1.25


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

# Access the variables
openai_api_key = os.getenv('OPENAI_API_KEY')
data_dir = os.getenv('DATA_DIR')
output_dir = os.getenv('OUTPUT_DIR')


In [3]:
from datasets import Dataset, load_dataset
import pandas as pd

answerset_json =f"{output_dir}/answerset.json"
answers_df = pd.read_json(answerset_json).drop(columns=["__index_level_0__"])
answers = Dataset.from_pandas(answers_df)
answers

/Users/cchase/git/github/cfchase/samsung-poc/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer', '__index_level_0__'],
    num_rows: 20
})

# Utility functions for LLM as judge

In [4]:
from langchain.prompts import PromptTemplate

STA_ANSWER_QUALITY_EVALUATION_PROMPT = PromptTemplate(
    template="""You are an evaluation system tasked with assessing the answer quality of a AI generated response in relation to the posed question and reference answer. Assess if the response is correct, accurate, and factual based on the reference answer. Evaluate the answer_quality as:
    - Score 1: The response is completely incorrect, inaccurate, and/or not factual.
    - Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
    - Score 3: The response is somewhat correct, accurate, and/or factual.
    - Score 4: The response is mostly correct, accurate, and factual.
    - Score 5: The response is completely correct, accurate, and factual.
    Here is the question: \n ------- \n {question} \n -------
    Here is model answer: \n ------- \n {answer} \n -------
    Here is the reference answer(may be very short and lack details or indirect, long and extractive):  \n ------- \n {reference_answer} \n ------- \n
    Assess the quality of model answer with respect to the Reference Answer, but do not penalize the model answer for adding details or give a direct answer to user question. Provide the quality level as a JSON object with two keys: 'reasoning' and 'answer_quality'.
    """,
    input_variables=["question", "answer", "reference_answer"],
)

In [5]:
from openai import OpenAI
import json

API_MAX_RETRY = 3
API_RETRY_SLEEP = 3
openai_client = OpenAI()

def chat_completion_openai(model, messages, temperature, max_tokens, api_dict=None):
    output = "OPENAI Error"
    for _ in range(API_MAX_RETRY):
        # try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            n=1,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        output = response.choices[0].message.content

    return output


def get_answer(messages: list, model: str, num_choices: int, max_tokens: int):

    temperature = 0.0

    outputs = []
    for i in range(num_choices):
        output = chat_completion_openai(model, messages, temperature, max_tokens)

    # Dump answers
    outputs.append(output)
    return outputs


def postprocess_judgement(output):
    try:
        json_output = json.loads(output)
        rating = json_output["answer_quality"]
        reason = json_output["reasoning"]

    except:
        rating = -1
        reason = "Judgement Error"

    return rating, reason


In [6]:
# Standalone Answer Quality Evaluation Prompt
import concurrent.futures
import concurrent
import tqdm


def run_sta_gpt_judgement(input_data_df, model="gpt-4-turbo", num_choices=1, max_tokens=1024, parallel=20):
    output_df = input_data_df.copy()
    ratings, reasons, raw_judgement_outputs = [], [], []
    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel) as executor:
        futures = []
        # iterate over the rows of the dataframe
        for index, row in output_df.iterrows():
            answer = row["answer"]
            question = row["question"]
            reference = row["ground_truth"]
            messages = [
                {"role": "user", "content": STA_ANSWER_QUALITY_EVALUATION_PROMPT.format(answer=answer, question=question, reference_answer=reference)}
            ]
            output = executor.submit(
                get_answer,
                messages,
                model,
                num_choices,
                max_tokens
            )
            futures.append(output)
    # Wait for all futures to complete
    concurrent.futures.wait(futures)

    for future in tqdm.tqdm(futures):
        output = future.result()
        rating, reason = postprocess_judgement(output[0])
        ratings.append(rating)
        reasons.append(reason)
        raw_judgement_outputs.append(output)

    # add the ratings and reasons and raw_output to the dataframe
    output_df["rating"] = ratings
    output_df["reason"] = reasons
    # output_df["Raw Judgement Output"] = raw_judgement_outputs
    # print(output_df.columns)

    return output_df

## GPT4-turbo eval

In [7]:
from datasets import Dataset, load_dataset
import pandas as pd

answerset_json =f"{output_dir}/answerset.json"
answers_df = pd.read_json(answerset_json).drop(columns=["__index_level_0__"])
answers_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer
0,How does Samsung SDS aim to upgrade their DevS...,[\n\nSOCIAL\n\nGOVERNANCE\n\n## **Sustainable ...,Samsung SDS aims to upgrade their DevSecOps le...,simple,[{'source': 'output/Samsung/md/document_257.md'}],True,Samsung SDS is working to upgrade their DevSec...
1,What is one of the factors that can be assesse...,[|-------------------------------------------|...,The work environment is one of the factors tha...,simple,[{'source': 'output/Samsung/md/document_787.md'}],True,One of the factors that can be assessed in the...
2,What does the Certification Support System at ...,[## **Regional Specialist Program**\n\nSamsung...,The Certification Support System at Samsung SD...,simple,[{'source': 'output/Samsung/md/document_248.md'}],True,The Certification Support System at Samsung SD...
3,What is the role of the Board of Directors in ...,[Samsung SDS designates a supporting departmen...,The role of the Board of Directors in Samsung ...,simple,[{'source': 'output/Samsung/md/document_299.md'}],True,The Board of Directors (BOD) in Samsung SDS is...
4,What is the purpose or function of the Steppin...,"[Self-reliance practice 2,600 people\n\nSteppi...",The purpose or function of the Stepping Stone ...,simple,[{'source': 'output/Samsung/md/document_463.md'}],True,The Stepping Stone of Hope Center is a program...
5,What is the purpose of the training provided f...,[Samsung SDS designates a supporting departmen...,Samsung SDS designates a supporting department...,simple,[{'source': 'output/Samsung/md/document_482.md'}],True,The purpose of the training provided for indep...
6,What is the purpose of the RE:CYCLE campaign c...,"[In line with the ESG business trend, many com...",The purpose of the RE:CYCLE campaign conducted...,simple,[{'source': 'output/Samsung/md/document_52.md'}],True,The RE:CYCLE campaign conducted by Samsung SDS...
7,What is the process for responding to personal...,[## **Process for Responding to Personal Data ...,In accordance with the Personal Information Pr...,simple,[{'source': 'output/Samsung/md/document_839.md'}],True,Samsung SDS has a system in place to respond p...
8,How many registered patents does Samsung SDS h...,"[As of the end of 2023, Samsung SDS operates o...","Total 4,758 persons (cumulative)",simple,[{'source': 'output/Samsung/md/document_189.md'}],True,"Samsung SDS has registered a total of 4,758 pa..."
9,What standards and guidelines does Samsung SDS...,[Suppliers must comply with all applicable law...,Samsung SDS requires suppliers to comply with ...,simple,[{'source': 'output/Samsung/md/document_674.md'}],True,Samsung SDS requires suppliers to comply with ...


In [8]:
import os

# model = "gpt-4-turbo"
# parallel = 10

model = "gpt-4"
parallel = 5

# model = "gpt-4o"
# parallel = 5

os.makedirs(model, exist_ok=True)

result_df = run_sta_gpt_judgement(
    answers_df,
    model=model,
    num_choices=1,
    max_tokens=1024,
    parallel=parallel)

result_df.to_csv(f"{output_dir}/{model}_eval_result.csv".replace(' ', '_').replace('-', '_'), 
                 index=False, 
                 header=True)
result_df


100%|██████████| 20/20 [00:00<00:00, 69442.12it/s]


,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer,rating,reason
0,How does Samsung SDS aim to upgrade their DevS...,[\n\nSOCIAL\n\nGOVERNANCE\n\n## **Sustainable ...,Samsung SDS aims to upgrade their DevSecOps le...,simple,[{'source': 'output/Samsung/md/document_257.md'}],True,Samsung SDS is working to upgrade their DevSec...,5,"The model answer is completely correct, accura..."
1,What is one of the factors that can be assesse...,[|-------------------------------------------|...,The work environment is one of the factors tha...,simple,[{'source': 'output/Samsung/md/document_787.md'}],True,One of the factors that can be assessed in the...,5,"The model answer is completely correct, accura..."
2,What does the Certification Support System at ...,[## **Regional Specialist Program**\n\nSamsung...,The Certification Support System at Samsung SD...,simple,[{'source': 'output/Samsung/md/document_248.md'}],True,The Certification Support System at Samsung SD...,5,"The model answer is completely correct, accura..."
3,What is the role of the Board of Directors in ...,[Samsung SDS designates a supporting departmen...,The role of the Board of Directors in Samsung ...,simple,[{'source': 'output/Samsung/md/document_299.md'}],True,The Board of Directors (BOD) in Samsung SDS is...,4,The model answer provides a comprehensive expl...
4,What is the purpose or function of the Steppin...,"[Self-reliance practice 2,600 people\n\nSteppi...",The purpose or function of the Stepping Stone ...,simple,[{'source': 'output/Samsung/md/document_463.md'}],True,The Stepping Stone of Hope Center is a program...,5,The model answer provides a detailed and compr...
5,What is the purpose of the training provided f...,[Samsung SDS designates a supporting departmen...,Samsung SDS designates a supporting department...,simple,[{'source': 'output/Samsung/md/document_482.md'}],True,The purpose of the training provided for indep...,4,The model answer is mostly correct and accurat...
6,What is the purpose of the RE:CYCLE campaign c...,"[In line with the ESG business trend, many com...",The purpose of the RE:CYCLE campaign conducted...,simple,[{'source': 'output/Samsung/md/document_52.md'}],True,The RE:CYCLE campaign conducted by Samsung SDS...,5,"The model answer is completely correct, accura..."
7,What is the process for responding to personal...,[## **Process for Responding to Personal Data ...,In accordance with the Personal Information Pr...,simple,[{'source': 'output/Samsung/md/document_839.md'}],True,Samsung SDS has a system in place to respond p...,5,"The model answer is completely correct, accura..."
8,How many registered patents does Samsung SDS h...,"[As of the end of 2023, Samsung SDS operates o...","Total 4,758 persons (cumulative)",simple,[{'source': 'output/Samsung/md/document_189.md'}],True,"Samsung SDS has registered a total of 4,758 pa...",5,"The model answer is completely correct, accura..."
9,What standards and guidelines does Samsung SDS...,[Suppliers must comply with all applicable law...,Samsung SDS requires suppliers to comply with ...,simple,[{'source': 'output/Samsung/md/document_674.md'}],True,Samsung SDS requires suppliers to comply with ...,5,The model answer is completely accurate and fa...
